# CIFAR Classification using Convolutional Nueral Networks

This notebook aims to illustrate all the different models used, their results and a discussion of the results.

### Explanation of run.py script

First, we need to import the packages being used. `Tensorflow` for building and processing the graph, `pycf` which has the data providers and models, and a few others.

In [1]:
# Import packages
import os
import datetime
import tensorflow as tf

import pycf.models as models
from pycf.utils import show_graph
from pycf.data_providers import CIFAR10DataProvider

# Set log level to suppress build warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

The data provider object can be easily defined as shown below. The `train_data` object is iterable over the batches. Hence, a forloop can be used for easy training.

In [2]:
# Load Training data
train_data = CIFAR10DataProvider(shape='2d')

In Tensorflow, the Graph contains all the operations. Hence, the model should be defined within the graph. Here the model `four_layer_fc_model` is used This model has four fully-connected layer. This is a very simple model which does not have enough modelling power for the task but it still can be used to confirm a correct setup.

In [3]:
# Create Graph
modelA = tf.Graph()

with modelA.as_default():
    metrics_ops, train_ops, data = models.fc6l()

    # Set logging files for tensorboard
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    train_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'train'), graph=tf.get_default_graph())
    
show_graph(modelA)

A Session is defined to train the model and the variables are initialized. Initialization is performed by running an init_op.

In [4]:
with modelA.as_default():
    sess = tf.Session()
    
    # Initialize variables
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

The model can now be trained to classify the different CIFAR images. Here we are training for the `num_epoch` variable number of epochs. There are two forloops one iterating over the epochs the other over the number of batches which are provided to the `feed_dict` of the run function. This `feed_dict` supplies the data to the tensorflow _inputs_ and _targets_ placeholders defined in the model. Later we shall look a method that the data loading is part of the graph in a queue fashion.

The result still have much to be desired as the error is still very high. We shall now look at other models which can help this get lower.

In [5]:
num_epoch = 20

with modelA.as_default():
    with sess:
        for epochs in range(num_epoch):
            running_err = 0
            running_acc = 0
            
            for batch_num, (input_batch, target_batch) in enumerate(train_data):    
                train_info, metrics = sess.run([train_ops, metrics_ops],
                                               feed_dict={data['inputs']: input_batch,
                                                          data['targets']: target_batch})
                
                train_writer.add_summary(train_info['summary'], epochs * train_data.num_batches + batch_num)
                running_err += metrics['error']
                running_acc += metrics['accuracy']
                

            running_err /= train_data.num_batches
            running_acc /= train_data.num_batches
            
            print('Epoch {0:02}\t| Average Error = {1:02.2f}\t| Average Accuracy = {2:.4f}'
                  .format(epochs + 1, running_err, running_acc))

Epoch 01	| Average Error = 2.51	| Average Accuracy = 0.1352
Epoch 02	| Average Error = 2.27	| Average Accuracy = 0.2037
Epoch 03	| Average Error = 2.13	| Average Accuracy = 0.2515
Epoch 04	| Average Error = 2.03	| Average Accuracy = 0.2869
Epoch 05	| Average Error = 1.95	| Average Accuracy = 0.3106
Epoch 06	| Average Error = 1.90	| Average Accuracy = 0.3323
Epoch 07	| Average Error = 1.85	| Average Accuracy = 0.3491
Epoch 08	| Average Error = 1.81	| Average Accuracy = 0.3654
Epoch 09	| Average Error = 1.77	| Average Accuracy = 0.3797
Epoch 10	| Average Error = 1.74	| Average Accuracy = 0.3883
Epoch 11	| Average Error = 1.71	| Average Accuracy = 0.4023
Epoch 12	| Average Error = 1.68	| Average Accuracy = 0.4123
Epoch 13	| Average Error = 1.66	| Average Accuracy = 0.4244
Epoch 14	| Average Error = 1.63	| Average Accuracy = 0.4309
Epoch 15	| Average Error = 1.61	| Average Accuracy = 0.4389
Epoch 16	| Average Error = 1.59	| Average Accuracy = 0.4463
Epoch 17	| Average Error = 1.57	| Averag

In [2]:
# Load Training data
train_data = CIFAR10DataProvider(shape='3d')
# Create Graph
modelA = tf.Graph()

with modelA.as_default():
    modelInfo = two_conv_two_fc_model()

    # Set logging files for tensorboard
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    train_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'train'), graph=modelA)
    
show_graph(modelA)

In [3]:
with modelA.as_default():
    sess = tf.Session()
    
    # Initialize variables
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    num_epoch = 20

with modelA.as_default():
    with sess:
        for epochs in range(num_epoch):
            running_error = 0
            for batch_num, (input_batch, target_batch) in enumerate(train_data):
                _, batch_error, summary = sess.run([modelInfo.get('train_op'),
                                                    modelInfo.get('error'),
                                                    modelInfo.get('summary_op')],

                                                   feed_dict={modelInfo.get('inputs'): input_batch,
                                                              modelInfo.get('targets'): target_batch})

                running_error += batch_error
                train_writer.add_summary(summary, epochs * train_data.num_batches + batch_num)

            running_error /= train_data.num_batches
            print('End of epoch {0}: Average epoch error = {1:.2f}'.format(epochs + 1, running_error))

End of epoch 1: Average epoch error = 24.84
End of epoch 2: Average epoch error = 16.12
End of epoch 3: Average epoch error = 11.09
End of epoch 4: Average epoch error = 5.90
End of epoch 5: Average epoch error = 3.45
End of epoch 6: Average epoch error = 2.78
End of epoch 7: Average epoch error = 2.54
End of epoch 8: Average epoch error = 2.44
End of epoch 9: Average epoch error = 2.39
End of epoch 10: Average epoch error = 2.36
End of epoch 11: Average epoch error = 2.34
End of epoch 12: Average epoch error = 2.33
End of epoch 13: Average epoch error = 2.33
End of epoch 14: Average epoch error = 2.32
End of epoch 15: Average epoch error = 2.32
End of epoch 16: Average epoch error = 2.32
End of epoch 17: Average epoch error = 2.31
End of epoch 18: Average epoch error = 2.31
End of epoch 19: Average epoch error = 2.31
End of epoch 20: Average epoch error = 2.31
